## **TMDB Movie Data Analysis using Pandas and APIs**


In [109]:
import os
import time
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv

### 1. **Fetching the movie data from the API**

In [110]:
load_dotenv()

TMDB_API_KEY = os.getenv("TMDB_API_KEY")

print(f"The TMDB API Key is fetched")

The TMDB API Key is fetched


In [111]:
# TMDB base URLs and Endpoints
BASE_URL = "https://api.themoviedb.org/3"
MOVIE_ENDPOINT = f"{BASE_URL}/movie"
CREDITS_ENDPOINT = f"{BASE_URL}/movie"

# Movie IDs to fetch
movie_ids = [
    0, 299534, 19995, 140607, 299536, 597, 135397, 420818,
    24428, 168259, 99861, 284054, 12445, 181808, 330457,
    351286, 109445, 321612, 260513
]

len(movie_ids), movie_ids[:5]


(19, [0, 299534, 19995, 140607, 299536])

In [112]:
def get_tmdb_json(url, params=None, verbose=True):
    """Get JSON from the API. Returns None if it fails."""
    
    if params is None:
        params = {}
    params["api_key"] = TMDB_API_KEY
    
    try:
        r = requests.get(url, params=params)
        r.raise_for_status()
        return r.json()
    except Exception as e:
        if verbose:
            print(f"Failed to get data from {url}")
        return None


In [113]:
def fetch_movie_with_credits(movie_id, sleep_time=0.2):
    """Get movie info and credits for an ID."""
    
    # Get movie details
    details_url = f"{MOVIE_ENDPOINT}/{movie_id}"
    details = get_tmdb_json(details_url)
    
    if not details:
        print(f"ID {movie_id}: Movie not found")
        return None
    
    # Get credits
    credits_url = f"{CREDITS_ENDPOINT}/{movie_id}/credits"
    credits = get_tmdb_json(credits_url)
    
    if not credits:
        credits = {"cast": [], "crew": []}
    
    # Build the record
    record = {}
    fields = [
        "id", "title", "tagline", "release_date", "genres",
        "belongs_to_collection", "original_language", "budget",
        "revenue", "production_companies", "production_countries",
        "vote_count", "vote_average", "popularity", "runtime",
        "overview", "spoken_languages", "poster_path", "status",
        "adult", "imdb_id", "original_title", "video", "homepage"
    ]
    
    for field in fields:
        record[field] = details.get(field)
    
    # Add credits
    record["cast"] = credits.get("cast", [])
    record["crew"] = credits.get("crew", [])
    
    # Wait a bit before next call
    time.sleep(sleep_time)
    
    return record

In [114]:
records = []

for m_id in movie_ids:
    print(f"Fetching movie_id = {m_id}")
    rec = fetch_movie_with_credits(m_id)
    if rec is not None:
        records.append(rec)

df_raw = pd.DataFrame(records)

df_raw.shape, df_raw.columns


Fetching movie_id = 0
Failed to get data from https://api.themoviedb.org/3/movie/0
ID 0: Movie not found
Fetching movie_id = 299534
Fetching movie_id = 19995
Fetching movie_id = 140607
Fetching movie_id = 299536
Fetching movie_id = 597
Fetching movie_id = 135397
Fetching movie_id = 420818
Fetching movie_id = 24428
Fetching movie_id = 168259
Fetching movie_id = 99861
Fetching movie_id = 284054
Fetching movie_id = 12445
Fetching movie_id = 181808
Fetching movie_id = 330457
Fetching movie_id = 351286
Fetching movie_id = 109445
Fetching movie_id = 321612
Fetching movie_id = 260513


((18, 26),
 Index(['id', 'title', 'tagline', 'release_date', 'genres',
        'belongs_to_collection', 'original_language', 'budget', 'revenue',
        'production_companies', 'production_countries', 'vote_count',
        'vote_average', 'popularity', 'runtime', 'overview', 'spoken_languages',
        'poster_path', 'status', 'adult', 'imdb_id', 'original_title', 'video',
        'homepage', 'cast', 'crew'],
       dtype='object'))

In [115]:
df_raw.head()

,id,title,tagline,release_date,genres,belongs_to_collection,original_language,budget,revenue,production_companies,...,spoken_languages,poster_path,status,adult,imdb_id,original_title,video,homepage,cast,crew
0,299534,Avengers: Endgame,Avenge the fallen.,2019-04-24,"[{'id': 12, 'name': 'Adventure'}, {'id': 878, ...","{'id': 86311, 'name': 'The Avengers Collection...",en,356000000,2799439100,"[{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZ...",...,"[{'english_name': 'English', 'iso_639_1': 'en'...",/bR8ISy1O9XQxqiy0fQFw2BX72RQ.jpg,Released,False,tt4154796,Avengers: Endgame,False,https://www.marvel.com/movies/avengers-endgame,"[{'adult': False, 'gender': 2, 'id': 3223, 'kn...","[{'adult': False, 'gender': 0, 'id': 3019687, ..."
1,19995,Avatar,Enter the world of Pandora.,2009-12-15,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","{'id': 87096, 'name': 'Avatar Collection', 'po...",en,237000000,2923706026,"[{'id': 444, 'logo_path': None, 'name': 'Dune ...",...,"[{'english_name': 'English', 'iso_639_1': 'en'...",/gKY6q7SjCkAU6FqvqWybDYgUKIF.jpg,Released,False,tt0499549,Avatar,False,https://www.avatar.com/movies/avatar,"[{'adult': False, 'gender': 2, 'id': 65731, 'k...","[{'adult': False, 'gender': 2, 'id': 58871, 'k..."
2,140607,Star Wars: The Force Awakens,Every generation has a story.,2015-12-15,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...","{'id': 10, 'name': 'Star Wars Collection', 'po...",en,245000000,2068223624,"[{'id': 1, 'logo_path': '/tlVSws0RvvtPBwViUyOF...",...,"[{'english_name': 'English', 'iso_639_1': 'en'...",/wqnLdwVXoBjKibFRR5U3y0aDUhs.jpg,Released,False,tt2488496,Star Wars: The Force Awakens,False,http://www.starwars.com/films/star-wars-episod...,"[{'adult': False, 'gender': 2, 'id': 3, 'known...","[{'adult': False, 'gender': 2, 'id': 491, 'kno..."
3,299536,Avengers: Infinity War,Destiny arrives all the same.,2018-04-25,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...","{'id': 86311, 'name': 'The Avengers Collection...",en,300000000,2052415039,"[{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZ...",...,"[{'english_name': 'English', 'iso_639_1': 'en'...",/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,Released,False,tt4154756,Avengers: Infinity War,False,https://www.marvel.com/movies/avengers-infinit...,"[{'adult': False, 'gender': 2, 'id': 3223, 'kn...","[{'adult': False, 'gender': 0, 'id': 3019687, ..."
4,597,Titanic,Nothing on earth could come between them.,1997-11-18,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",None,en,200000000,2264162353,"[{'id': 4, 'logo_path': '/jay6WcMgagAklUt7i9Eu...",...,"[{'english_name': 'English', 'iso_639_1': 'en'...",/9xjZS2rlVxm8SFx8kPC3aIGCOYQ.jpg,Released,False,tt0120338,Titanic,False,https://www.paramountmovies.com/movies/titanic,"[{'adult': False, 'gender': 2, 'id': 6193, 'kn...","[{'adult': False, 'gender': 2, 'id': 2710, 'kn..."


In [116]:
df_raw.dtypes

id                         int64
title                     object
tagline                   object
release_date              object
genres                    object
belongs_to_collection     object
original_language         object
budget                     int64
revenue                    int64
production_companies      object
production_countries      object
vote_count                 int64
vote_average             float64
popularity               float64
runtime                    int64
overview                  object
spoken_languages          object
poster_path               object
status                    object
adult                       bool
imdb_id                   object
original_title            object
video                       bool
homepage                  object
cast                      object
crew                      object
dtype: object

### 2. **DATA CLEANING AND PREPROCESSING**
---
#### **Data Preparation and Cleaning**

In [117]:
# Dropping unnecessary columns
movie_df = df_raw.copy()
cols_to_drop = ['adult', 'imdb_id', 'original_title', 'video', 'homepage']
movie_df = movie_df.drop(columns=[c for c in cols_to_drop if c in movie_df.columns])

movie_df.columns


Index(['id', 'title', 'tagline', 'release_date', 'genres',
       'belongs_to_collection', 'original_language', 'budget', 'revenue',
       'production_companies', 'production_countries', 'vote_count',
       'vote_average', 'popularity', 'runtime', 'overview', 'spoken_languages',
       'poster_path', 'status', 'cast', 'crew'],
      dtype='object')

In [118]:
def extract_collection_name(x):
    """Extract collection name from belongs_to_collection"""
    if isinstance(x, dict) and 'name' in x:
        return x.get('name')
    return np.nan

movie_df["belongs_to_collection"] = movie_df["belongs_to_collection"].apply(extract_collection_name)
movie_df["belongs_to_collection"].value_counts(dropna=False)

belongs_to_collection
The Avengers Collection                4
Star Wars Collection                   2
NaN                                    2
Jurassic Park Collection               2
Frozen Collection                      2
Avatar Collection                      1
The Lion King (Reboot) Collection      1
The Fast and the Furious Collection    1
Black Panther Collection               1
Harry Potter Collection                1
The Incredibles Collection             1
Name: count, dtype: int64

In [119]:
def extract_names_from_list(lst, key='name'):
        """Extract names from list of dictionaries and joins them into a string separated with | for clarity"""
        if isinstance(lst, list) and len(lst) > 0:
            names = [d.get(key, '') for d in lst if isinstance(d, dict)]
            return '|'.join(filter(None, names))
        return np.nan

json_cols = [
    "genres",
    "spoken_languages",
    "production_countries",
    "production_companies",
]


In [120]:

for col in json_cols:
    if col in movie_df.columns:
        movie_df[col] = movie_df[col].apply(extract_names_from_list)

movie_df[["genres", "spoken_languages", "production_countries", "production_companies"]].value_counts(dropna=False)


genres                                     spoken_languages                                   production_countries                     production_companies                                                                
Action|Adventure|Animation|Family          English                                            United States of America                 Pixar                                                                                   1
Action|Adventure|Fantasy|Science Fiction   English|Español                                    United States of America|United Kingdom  Dune Entertainment|Lightstorm Entertainment|20th Century Fox|Ingenious Film Partners    1
Family|Fantasy|Romance                     English|Français                                   United States of America                 Walt Disney Pictures|Mandeville Films                                                   1
Family|Animation|Adventure|Comedy|Fantasy  English                                            United Stat

#### **Cast and Directors Information**

In [ ]:
def get_cast_info(cast_list):
    """Count number of cast members, and show all of their names"""
    if isinstance(cast_list, list):
        names = [person.get('name') for person in cast_list
                 if isinstance(person, dict) and person.get('name')]
        cast_size = len(names)
        if cast_size == 0:
            return np.nan, 0
        cast_str = "|".join(names)
        return cast_str, cast_size
    return np.nan, 0

def get_director(crew_list):
    """Extract director name from crew"""
    if isinstance(crew_list, list):
        for person in crew_list:
            if isinstance(person, dict) and person.get('job') == 'Director':
                return person.get('name')
    return np.nan

def get_crew_size(crew_list):
    """Count number of crew members"""
    if isinstance(crew_list, list):
        return len(crew_list)
    return 0

In [122]:
# Cast
cast_info = movie_df["cast"].apply(lambda x: pd.Series(get_cast_info(x)))
movie_df["cast"] = cast_info[0]       
movie_df["cast_size"] = cast_info[1]

#Crew
movie_df["director"] = movie_df["crew"].apply(get_director)
movie_df["crew_size"] = movie_df["crew"].apply(get_crew_size)

# Now we can drop the original column
movie_df = movie_df.drop(columns=["crew"])
movie_df[["title", "cast", "cast_size", "director", "crew_size"]].head()


,title,cast,cast_size,director,crew_size
0,Avengers: Endgame,Robert Downey Jr.|Chris Evans|Mark Ruffalo|Chr...,105,Anthony Russo,607
1,Avatar,Sam Worthington|Zoe Saldaña|Sigourney Weaver|S...,65,James Cameron,990
2,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,183,J.J. Abrams,262
3,Avengers: Infinity War,Robert Downey Jr.|Chris Evans|Chris Hemsworth|...,69,Anthony Russo,733
4,Titanic,Leonardo DiCaprio|Kate Winslet|Billy Zane|Kath...,116,James Cameron,262


#### **Converting data types to more conventional ones**

In [123]:
# Numeric columns
numeric_cols = ["budget", "revenue", "id", "popularity", "vote_count", "vote_average", "runtime"]

for col in numeric_cols:
    if col in movie_df.columns:
        movie_df[col] = pd.to_numeric(movie_df[col], errors="coerce")

# Release_date to datetime
if "release_date" in movie_df.columns:
    movie_df["release_date"] = pd.to_datetime(movie_df["release_date"], errors="coerce")

movie_df.dtypes

id                                int64
title                            object
tagline                          object
release_date             datetime64[ns]
genres                           object
belongs_to_collection            object
original_language                object
budget                            int64
revenue                           int64
production_companies             object
production_countries             object
vote_count                        int64
vote_average                    float64
popularity                      float64
runtime                           int64
overview                         object
spoken_languages                 object
poster_path                      object
status                           object
cast                             object
cast_size                         int64
director                         object
crew_size                         int64
dtype: object

#### **Replacing Unrealistic Values**

Handling 0 values and converting to millions. Chose to convert the these to `NaN` or missing in regard of keeping authenticity and not adding things we are not sure of.

In [124]:
for col in ["budget", "revenue", "runtime"]:
    if col in movie_df.columns:
        n_zeros = (movie_df[col] == 0).sum()
        print(f"{col}: replacing {n_zeros} zeros with NaN")
        movie_df[col] = movie_df[col].replace(0, np.nan)


# Million conversion
movie_df["budget_musd"] = movie_df["budget"] / 1_000_000
movie_df["revenue_musd"] = movie_df["revenue"] / 1_000_000

movie_df[["title", "budget", "budget_musd", "revenue", "revenue_musd", "runtime"]].head()


budget: replacing 0 zeros with NaN
revenue: replacing 0 zeros with NaN
runtime: replacing 0 zeros with NaN


,title,budget,budget_musd,revenue,revenue_musd,runtime
0,Avengers: Endgame,356000000,356.0,2799439100,2799.439100,181
1,Avatar,237000000,237.0,2923706026,2923.706026,162
2,Star Wars: The Force Awakens,245000000,245.0,2068223624,2068.223624,136
3,Avengers: Infinity War,300000000,300.0,2052415039,2052.415039,149
4,Titanic,200000000,200.0,2264162353,2264.162353,194


Appears that we had nothing to worry about for zero values in these columns! Just cautious


In [125]:
movie_df = movie_df.drop(columns=["budget", "revenue"]) # We don't need these anymore

#### **Cleaning vote_count and other text fields**

In [126]:
mask_zero_votes = (movie_df["vote_count"] == 0)
movie_df.loc[mask_zero_votes, "vote_average"] = np.nan

# Cleaning placeholder text in 'overview' and 'tagline'
placeholders = ["No Data", "No overview found.", "", " ", "N/A", None]

for col in ["overview", "tagline"]:
    if col in movie_df.columns:
        movie_df[col] = movie_df[col].replace(placeholders, np.nan)


#### **Removing Duplicates & Invalid Rows**
Let us remove duplicates in crucial columns like id. Also chose to keep only rows whose missing values don't exceed 10 or those with either no id or title.


In [127]:
# Remove duplicates by movie ID
movie_df = movie_df.drop_duplicates(subset="id")

# Drop rows missing ID or title
movie_df = movie_df.dropna(subset=["id", "title"])
movie_df = movie_df.dropna(thresh=10)
movie_df.shape


(18, 23)

The dataframe appears unchanged which means that we have no duplicates, missing titles or ids, or with more than 10 missing values. That's a good thing.

#### **Keep only “Released” movies and drop status**

After keeping only released  movies, we know their status so no need for that columns anymore!

In [128]:
if "status" in movie_df.columns:
    print("Status counts before filtering:")
    print(movie_df["status"].value_counts(dropna=False))

    movie_df = movie_df[movie_df["status"] == "Released"].copy()
    movie_df = movie_df.drop(columns=["status"], errors="ignore")

Status counts before filtering:
status
Released    18
Name: count, dtype: int64


Turns out no unreleased movies anyway! So our data is still intact.

#### **Reorder Columns and Reset Index**
Now we can follow the order given

In [129]:
final_columns = [
    'id', 'title', 'tagline', 'release_date', 'genres','belongs_to_collection', 
    'original_language', 'budget_musd', 'revenue_musd','production_companies', 
    'production_countries','vote_count', 'vote_average', 'popularity', 'runtime',
    'overview', 'spoken_languages', 'poster_path','cast', 'cast_size', 'director', 'crew_size'
]
movie_df = movie_df[final_columns].reset_index(drop = True)
movie_df.head()

,id,title,tagline,release_date,genres,belongs_to_collection,original_language,budget_musd,revenue_musd,production_companies,...,vote_average,popularity,runtime,overview,spoken_languages,poster_path,cast,cast_size,director,crew_size
0,299534,Avengers: Endgame,Avenge the fallen.,2019-04-24,Adventure|Science Fiction|Action,The Avengers Collection,en,356.0,2799.439100,Marvel Studios,...,8.237,14.0697,181,After the devastating events of Avengers: Infi...,English|日本語,/bR8ISy1O9XQxqiy0fQFw2BX72RQ.jpg,Robert Downey Jr.|Chris Evans|Mark Ruffalo|Chr...,105,Anthony Russo,607
1,19995,Avatar,Enter the world of Pandora.,2009-12-15,Action|Adventure|Fantasy|Science Fiction,Avatar Collection,en,237.0,2923.706026,Dune Entertainment|Lightstorm Entertainment|20...,...,7.594,35.8342,162,"In the 22nd century, a paraplegic Marine is di...",English|Español,/gKY6q7SjCkAU6FqvqWybDYgUKIF.jpg,Sam Worthington|Zoe Saldaña|Sigourney Weaver|S...,65,James Cameron,990
2,140607,Star Wars: The Force Awakens,Every generation has a story.,2015-12-15,Adventure|Action|Science Fiction,Star Wars Collection,en,245.0,2068.223624,Lucasfilm Ltd.|Bad Robot,...,7.300,7.3413,136,Thirty years after defeating the Galactic Empi...,English,/wqnLdwVXoBjKibFRR5U3y0aDUhs.jpg,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,183,J.J. Abrams,262
3,299536,Avengers: Infinity War,Destiny arrives all the same.,2018-04-25,Adventure|Action|Science Fiction,The Avengers Collection,en,300.0,2052.415039,Marvel Studios,...,8.235,20.5218,149,As the Avengers and their allies have continue...,English,/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,Robert Downey Jr.|Chris Evans|Chris Hemsworth|...,69,Anthony Russo,733
4,597,Titanic,Nothing on earth could come between them.,1997-11-18,Drama|Romance,NaN,en,200.0,2264.162353,Paramount Pictures|20th Century Fox|Lightstorm...,...,7.903,27.2152,194,101-year-old Rose DeWitt Bukater tells the sto...,English|Français|Deutsch|svenska|Italiano|Pусский,/9xjZS2rlVxm8SFx8kPC3aIGCOYQ.jpg,Leonardo DiCaprio|Kate Winslet|Billy Zane|Kath...,116,James Cameron,262


We run into no errors so, all the columns did exist and now we have them in the order we want. We can proceed to KPI calculations and Analysis.
